In [1]:
# ==========================================
# Cell 0 : 추가 패키지 설치
# ==========================================

# 설치를 했는데 셀 3번 7번에서 작동이 안 될 경우 터미널에서 skn17_final_env 커널이 연결되어 있는 상황에서 설치
!pip install audio_separator
!pip install -U bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached julius-0.2.7.tar.gz (59 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
  Using cached soundfile-0.13.1-py2.py3-none-manylinux_2_28_x86_64.whl.metadata (16 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached cython-3.2.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (5.0 kB)
  Using cached audioread-3.1.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached pooch-1.8

In [2]:
# ==========================================
# Cell 1: 경로 설정 및 토큰 활성화
# ==========================================

from pathlib import Path
import os, sys
import json

# ---- 프로젝트 루트 ----
PROJECT_ROOT = Path("/workspace/baseball_pipeline")
os.chdir(PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

# ---- 데이터 디렉토리 ----
from src import DATA_DIR, FAISS_DIR, FISH_ROOT

INPUT_VIDEO_DIR = DATA_DIR / "input_videos"
STT_RAW_DIR = DATA_DIR / "stt_raw"
STT_SEG_DIR = DATA_DIR / "stt_segments"
LLM_OUT_DIR = DATA_DIR / "llm_outputs"
TTS_AUDIO_DIR = DATA_DIR / "tts_audio"
OUTPUT_VIDEO_DIR = DATA_DIR / "output_videos"
AUDIO_ROOT = DATA_DIR / "audio_separator"
FRAMES_ROOT = DATA_DIR / "frames"
SRC_ROOT = PROJECT_ROOT / "src"

if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

# 디렉토리 생성
for d in (DATA_DIR, INPUT_VIDEO_DIR, STT_RAW_DIR, STT_SEG_DIR, 
          LLM_OUT_DIR, TTS_AUDIO_DIR, OUTPUT_VIDEO_DIR, AUDIO_ROOT, 
          FRAMES_ROOT, FAISS_DIR, SRC_ROOT):
    d.mkdir(parents=True, exist_ok=True)

print("\n✅ 디렉토리 생성 완료")

# ---- API 토큰 설정 ----
CLOVA_INVOKE_URL = ""
CLOVA_SECRET_KEY = ""

HF_TOKEN = ""
# OPENAI_API_KEY = "sk-proj-..."  # 필요시 입력

if HF_TOKEN and "xxx" not in HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

# if OPENAI_API_KEY:
#     os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 토큰 설정 완료\n")

PROJECT_ROOT: /workspace/baseball_pipeline

✅ 디렉토리 생성 완료
✅ API 토큰 설정 완료



In [3]:
from src.google_mp4_download import download_gdrive_video

# ====== 여기에 구글 드라이브 링크 입력 ======
gdrive_url = "https://drive.google.com/file/d/1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn/view?usp=sharing"
VIDEO_NAME = "체코_대한민국_wbc.mp4"

local_video_path = download_gdrive_video(gdrive_url, dest_name=VIDEO_NAME)
video_stem = Path(VIDEO_NAME).stem

print(f"\n✅ 영상 다운로드 완료")
print(f"  video_stem: {video_stem}")
print(f"  경로: {local_video_path}\n")

[GDRIVE] file_id: 1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn
[GDRIVE] url    : https://drive.google.com/uc?id=1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn
[GDRIVE] output : /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4


Downloading...
From (original): https://drive.google.com/uc?id=1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn
From (redirected): https://drive.google.com/uc?id=1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn&confirm=t&uuid=69f2a8f3-c11d-49e4-b630-cc7d149a2e9b
To: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4
100%|██████████| 1.29G/1.29G [00:25<00:00, 50.1MB/s]


[GDRIVE] 다운로드 완료: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4

✅ 영상 다운로드 완료
  video_stem: 체코_대한민국_wbc
  경로: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4



In [ ]:
# ==========================================
# Cell 3: audio_separator 음성 분리
# ==========================================
from src.audio_separator import separate_audio_sota

track_dict = separate_audio_sota(
    video_path=str(local_video_path),
    output_dir=str(AUDIO_ROOT),
    device="cuda"
)

vocals_path = track_dict["vocals"]
no_vocals_path = track_dict["no_vocals"]

print(f"\n✅ 음성 분리 완료! (SOTA Performance)")
print(f"  - 해설(Vocals)    : {vocals_path}")
print(f"  - 현장음(No Vocals): {no_vocals_path}\n")


2025-12-08 03:18:23,676 - INFO - separator - Separator version 0.40.0 instantiating with output_dir: /workspace/skn17_final_runpod_code/baseball_pipeline/data/audio_separator, output_format: wav
2025-12-08 03:18:23,677 - INFO - separator - Using model directory from model_file_dir parameter: /workspace/skn17_final_runpod_code/baseball_pipeline/data/audio_separator/models
2025-12-08 03:18:23,680 - INFO - separator - Operating System: Linux #63~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 22 19:00:15 UTC 2
2025-12-08 03:18:23,681 - INFO - separator - System: Linux Node: f75de6caa909 Release: 6.8.0-60-generic Machine: x86_64 Proc: x86_64
2025-12-08 03:18:23,681 - INFO - separator - Python Version: 3.10.19
2025-12-08 03:18:23,682 - INFO - separator - PyTorch Version: 2.5.1+cu121
2025-12-08 03:18:23,737 - INFO - separator - FFmpeg installed: ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
2025-12-08 03:18:23,749 - INFO - separator - ONNX Runtime CPU package install


 오디오 분리 시작: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4
  사용 모델: model_bs_roformer_ep_317_sdr_12.9755.ckpt (BS-RoFormer-ViperX)
  실행 장치: cuda


2025-12-08 03:18:23,903 - INFO - separator - CUDA is available in Torch, setting Torch device to CUDA
2025-12-08 03:18:23,904 - WARNING - separator - CUDAExecutionProvider not available in ONNXruntime, so acceleration will NOT be enabled
2025-12-08 03:18:23,905 - INFO - separator - Loading model model_bs_roformer_ep_317_sdr_12.9755.ckpt...


  >> 모델 로딩 중... (최초 실행 시 다운로드에 시간이 소요됩니다)


28.3kiB [00:00, 1.43MiB/s]                  
100%|██████████| 639M/639M [00:13<00:00, 48.0MiB/s] 
100%|██████████| 2.27k/2.27k [00:00<00:00, 217kiB/s]
2025-12-08 03:18:49,418 - INFO - mdxc_separator - MDXC Separator initialisation complete
2025-12-08 03:18:49,420 - INFO - separator - Roformer loading stats: {'new_implementation_success': 1, 'total_failures': 0}
2025-12-08 03:18:49,420 - INFO - separator - Load model duration: 00:00:25
2025-12-08 03:18:49,427 - INFO - separator - Processing file: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4
2025-12-08 03:18:49,428 - INFO - separator - Starting separation process for audio_file_path: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4
2025-12-08 03:18:49,565 - WARNING - common_separator - Could not read audio file info, defaulting to 16-bit output: Error opening '/workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4': Format not 

  >> 분리 작업 수행 중...


  6%|▌         | 92/1576 [01:26<23:17,  1.06it/s]

In [ ]:
# ==========================================
# Cell 4: STT (Clova Speech API)
# ==========================================
from pathlib import Path
import json

from src.stt_pipeline import run_stt_pipeline
from src.stt_event_splitter import stt_json_to_event_sets

print(f"[STT] Clova STT 시작")
print(f"  입력: {vocals_path}")

STT_KEYWORD_XLSX = PROJECT_ROOT / "stt.xlsx"
if STT_KEYWORD_XLSX.exists():
    xlsx_path = STT_KEYWORD_XLSX
    use_domain = False
    print("  키워드: stt.xlsx 사용 (엑셀 부스팅)")
else:
    xlsx_path = None
    use_domain = True
    print("  키워드: 도메인 부스팅만 사용")

# ---- STT 파이프라인 실행 ----
timeline_json_path = run_stt_pipeline(
    audio_path=vocals_path,
    invoke_url=CLOVA_INVOKE_URL,
    secret_key=CLOVA_SECRET_KEY,
    stt_raw_dir=STT_RAW_DIR,
    stt_seg_dir=STT_SEG_DIR,
    xlsx_keywords_path=xlsx_path,
    use_domain_boostings=use_domain,
    speaker_count_min=2,
    speaker_count_max=3,
    save_raw_json=True,
    pause_thresh_ms=50000,  # 필요시 조절
)

print(f"\n✅ STT 완료!")
print(f"timeline.json : {timeline_json_path}\n")

In [ ]:
# ==========================================
# Cell 5: STT 데이터 전처리 (이벤트 세트)
# ==========================================
from src.stt_event_splitter import stt_json_to_event_sets

timeline_json_stem = timeline_json_path.stem
# 같은 디렉터리에 *_output.json 으로 저장
json_after_split_path = timeline_json_path.with_name(f"{timeline_json_stem}_set_split.json")

# 입력 JSON 로드
with timeline_json_path.open("r", encoding="utf-8") as f:
    stt_json = json.load(f)

# 이벤트 세트 변환
event_sets = stt_json_to_event_sets(
    stt_json,
    caster_gap=10.0,   # 필요시 튜닝
    silence_gap=2.0,  # 필요시 튜닝
)

print(f"이벤트 세트 개수: {len(event_sets)}")

# 출력 JSON 저장
with json_after_split_path.open("w", encoding="utf-8") as f:
    json.dump(event_sets, f, ensure_ascii=False, indent=2)

print(f"저장 완료: {json_after_split_path.resolve()}")

In [ ]:
# ==========================================
# Cell 6: 영상 이미지 추출
# ==========================================

import json

from src.image_extraction import capture_frames_for_sets

# 세트 json 로드
with json_after_split_path.open("r", encoding="utf-8") as f:
    sets = json.load(f)   # 리스트 형태여야 함

print(f"세트 개수: {len(sets)}")

# 5) 세트의 set_start_sec 기준으로 이미지 추출
results = capture_frames_for_sets(
    video_path=local_video_path,
    sets=sets,
    output_dir=FRAMES_ROOT
)

# 6) 요약 출력
success_count = sum(1 for r in results if r["success"])
print(f"성공적으로 저장된 이미지 수: {success_count}")


In [ ]:
# ==========================================
# Cell 7: VLM 스코어보드 추출 (최적화 버전)
# ==========================================

from pathlib import Path

from src.vlm_scoreboard import (
    load_scoreboard_model_and_processor,
    attach_scoreboard_to_sets,
)

# ==========================
# 모델 / 프로세서 로드
# ==========================

vlm_model, vlm_processor = load_scoreboard_model_and_processor()


# ==========================
# scoreboard 파이프라인 실행
# ==========================
json_after_split_stem = json_after_split_path.stem
# 같은 디렉터리에 *_output.json 으로 저장
scoreboard_json_path = json_after_split_path.with_name(f"{json_after_split_stem}_scoreboard.json")

updated_sets = attach_scoreboard_to_sets(
    json_after_split_path=json_after_split_path,
    output_json_path=scoreboard_json_path,
    frames_root=FRAMES_ROOT,
    video_path=local_video_path,
    model=vlm_model,
    processor=vlm_processor,
    retry_if_all_null=False,   # all-null이면 +2초 재시도
    retry_offset_sec=2.0,
)

print(f"총 세트 수: {len(updated_sets)}")
print(f"저장 완료: {scoreboard_json_path.resolve()}")

# 일부만 눈으로 확인
for s in updated_sets[:5]:
    print("=" * 80)
    print("set_id:", s["set_id"])
    print("set_start_sec:", s["set_start_sec"])
    print("scoreboard:", s["scoreboard"])

In [ ]:
from pathlib import Path
import json

# from pakchanho_commentary_generator import (
#     load_pakchanho_model,
#     generate_analyst_for_all_sets,
# )

from src.llm_generator import (
    load_pakchanho_model,
    generate_analyst_for_all_sets,
)

# Pakchanho LLM 적용 후 저장할 경로
json_llm_output_path = LLM_OUT_DIR / "vocals_timeline_set_split_scoreboard_pakchanho.json"

# 1) 모델 로드
model, tokenizer = load_pakchanho_model(
    base_model_name="kakaocorp/kanana-1.5-8b-instruct-2505",
    lora_model_id="SeHee8546/kanana-1.5-8b-pakchanho-lora-v2",
    load_in_4bit=True,
)

# 2) 세트별 analyst_text 재생성
game_title = "2025 KBO 준플레이오프 4차전 삼성 vs SSG"  # 경기 정보는 상황에 맞게 바꾸면 됨

result_sets = generate_analyst_for_all_sets(
    json_in_path=scoreboard_json_path,
    json_out_path=json_llm_output_path,
    model=model,
    tokenizer=tokenizer,
    game_title=game_title,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3,
    base_max_new_tokens=512,
)

print(f"총 세트 수: {len(result_sets)}")
print(f"저장 완료: {json_llm_output_path.resolve()}")

# 3) 샘플 몇 개 확인
for row in result_sets[:5]:
    print("=" * 80)
    print("set_id:", row["set_id"])
    print("caster_text:", row["caster_text"])
    print("analyst_text:", row["analyst_text"])

In [ ]:
# ==========================================
# JSON 기반 TTS 전체 파이프라인
# ==========================================

from pathlib import Path
import pandas as pd

from src.json_tts_pipeline import run_full_tts_pipeline_from_json


# 이 영상에 대한 정보들
video_stem = local_video_path.stem

# LLM을 통과한 최종 세트 JSON (지금 올려준 파일)
# 현재는 이름이 "vocals_timeline_set_split_scoreboard_pakchanho (4).json" 이니까
# 1) 파일명을 위 규칙으로 바꾸거나
# 2) 그냥 정확한 이름을 직접 Path 로 넣어도 됩니다.
# json_sets_path = DATA_DIR / "llm_outputs" / "vocals_timeline_set_split_scoreboard_pakchanho (4).json"

# Fish-Speech API 설정
FISH_API_URL = "http://127.0.0.1:8080/v1/tts"

CASTER_REF_WAVS = [DATA_DIR / "tts_refs" / "caster_prompt_1.wav"]
ANALYST_REF_WAVS = [DATA_DIR / "tts_refs" / "analyst_pakchanho_prompt_1.wav"]

print(f"[TTS] JSON 기반 TTS 파이프라인 시작")
print(f"  JSON 세트 파일: {json_llm_output_path}")
print(f"  원본 영상: {local_video_path}")
print(f"  캐스터 참조: {CASTER_REF_WAVS}")
print(f"  해설 참조: {ANALYST_REF_WAVS}")

try:
    final_tts_wav, aligned_csv, tts_csv_with_paths = run_full_tts_pipeline_from_json(
        json_sets_path=json_llm_output_path,
        video_path=local_video_path,
        caster_ref_wavs=CASTER_REF_WAVS,
        analyst_ref_wavs=ANALYST_REF_WAVS,
        fish_api_url=FISH_API_URL,
        # 아래 파라미터들은 기존 셋업 그대로 사용 (필요하면 튜닝 가능)
        min_text_chars=2,
        merge_same_role=True,
        merge_gap_thresh_sec=0.25,
        merge_short_thresh_sec=1.0,
        min_gap_sec=0.02,
        caster_extra_ratio=0.2,
        analyst_extra_ratio=2.0,
        max_analyst_expand_sec=7.0,
        analyst_priority_min_overlap_sec=0.5,
        min_gap_ms=60,
        tail_margin_ms=80,
        caster_max_speedup=1.3,
        analyst_max_speedup=1.8,
    )

    print("\n✅ JSON → TTS → 정렬 → WSOLA 전체 완료!")
    print(f"  - TTS CSV(with paths): {tts_csv_with_paths}")
    print(f"  - 정렬 CSV: {aligned_csv}")
    print(f"  - 최종 TTS 타임라인 wav: {final_tts_wav}\n")

    # 통계 출력 (선택)
    tts_df = pd.read_csv(tts_csv_with_paths)
    print("📊 TTS 통계:")
    print(f"  - 총 발화 수: {len(tts_df)}")
    print(f"  - TTS 성공: {tts_df['tts_wav_path'].notna().sum()}개")
    print(f"  - TTS 실패: {tts_df['tts_wav_path'].isna().sum()}개")

except Exception as e:
    print(f"\n❌ JSON 기반 TTS 파이프라인 실패: {e}")
    import traceback
    traceback.print_exc()
    raise


In [ ]:
# ==========================================
# Cell 12: 최종 영상 인코딩
# ==========================================
import subprocess
from pathlib import Path

# ⭐ 출력 디렉토리 수정
OUTPUT_VIDEO_DIR = DATA_DIR / "output_videos"
OUTPUT_VIDEO_DIR.mkdir(parents=True, exist_ok=True)

# ⭐ 경로 명확하게 정의
TTS_AUDIO_DIR = DATA_DIR / "tts_audio" / video_stem
AUDIO_SEPARATOR_DIR = DATA_DIR / "audio_separator"

# TTS 타임라인 wav (최종 생성된 TTS 음성)
final_tts_wav = TTS_AUDIO_DIR / f"{video_stem}.tts_timeline.wav"

# Audio Separator에서 분리한 배경음
no_vocals_path = AUDIO_SEPARATOR_DIR / "no_vocals.wav"

def get_duration(file_path: Path) -> float:
    """파일의 길이를 초 단위로 반환"""
    cmd = [
        'ffprobe', '-v', 'error',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        str(file_path)
    ]
    result = subprocess.run(cmd, capture_output=True, encoding='utf-8', errors='replace', check=True)
    return float(result.stdout.strip())

def merge_audio_and_encode_video(
    original_video_path: Path,
    tts_vocals_path: Path,
    bg_no_vocals_path: Path,
    output_video_path: Path,
    tts_volume: float = 1.0,
    bg_volume: float = 0.7,
) -> Path:
    """
    1) TTS vocals + 배경음(no_vocals) 믹싱
    2) 원본 비디오와 합쳐서 최종 영상 생성
    """
    output_video_path.parent.mkdir(parents=True, exist_ok=True)
    
    # 파일 존재 여부 먼저 확인
    print("\n" + "=" * 80)
    print("📁 파일 존재 여부 확인")
    print("=" * 80)
    
    files_to_check = {
        "원본 비디오": original_video_path,
        "TTS 오디오": tts_vocals_path,
        "배경음": bg_no_vocals_path,
    }
    
    for name, path in files_to_check.items():
        if path.exists():
            size_mb = path.stat().st_size / (1024**2)
            print(f"✅ {name}: {path.name} ({size_mb:.2f} MB)")
        else:
            print(f"❌ {name}: {path}")
            raise FileNotFoundError(f"{name} 파일이 존재하지 않습니다: {path}")
    
    print("=" * 80 + "\n")
    
    # 파일 길이 확인
    try:
        video_duration = get_duration(original_video_path)
        tts_duration = get_duration(tts_vocals_path)
        bg_duration = get_duration(bg_no_vocals_path)
        
        print(f"[INFO] 📹 비디오 길이: {video_duration:.2f}초 ({int(video_duration//60)}분 {int(video_duration%60)}초)")
        print(f"[INFO] 🎤 TTS 오디오 길이: {tts_duration:.2f}초 ({int(tts_duration//60)}분 {int(tts_duration%60)}초)")
        print(f"[INFO] 🎵 배경음 길이: {bg_duration:.2f}초 ({int(bg_duration//60)}분 {int(bg_duration%60)}초)")
        
        # 길이 차이가 크면 경고
        if abs(tts_duration - bg_duration) > 5.0:
            print(f"[WARNING] ⚠️ TTS와 배경음 길이 차이: {abs(tts_duration - bg_duration):.2f}초")
        if abs(video_duration - tts_duration) > 5.0:
            print(f"[WARNING] ⚠️ 비디오와 TTS 길이 차이: {abs(video_duration - tts_duration):.2f}초")
            
    except Exception as e:
        print(f"[WARNING] 파일 길이 확인 실패: {e}")
        print("[INFO] 길이 확인 없이 진행합니다...")
    
    print()
    
    # 더 안전한 FFmpeg 명령어 (2단계 처리)
    # Step 1: 오디오 믹싱
    mixed_audio_path = output_video_path.parent / f"{output_video_path.stem}_mixed_audio.wav"
    
    audio_cmd = [
        'ffmpeg', '-y',
        '-i', str(tts_vocals_path),
        '-i', str(bg_no_vocals_path),
        '-filter_complex',
        f'[0:a]volume={tts_volume}[a1];'
        f'[1:a]volume={bg_volume}[a2];'
        f'[a1][a2]amix=inputs=2:duration=longest:normalize=0[aout]',
        '-map', '[aout]',
        '-ar', '44100',  # 샘플레이트 고정
        '-ac', '2',       # 스테레오
        str(mixed_audio_path)
    ]
    
    print("[STEP 1] 🎵 오디오 믹싱 중...")
    
    result = subprocess.run(audio_cmd, capture_output=True, encoding='utf-8', errors='replace')
    
    if result.returncode != 0:
        print(f"\n❌ 오디오 믹싱 실패!")
        print(f"Return code: {result.returncode}")
        print(f"\nStderr:\n{result.stderr}")
        raise subprocess.CalledProcessError(result.returncode, audio_cmd, result.stdout, result.stderr)
    
    print(f"[STEP 1] ✅ 오디오 믹싱 완료!")
    
    # Step 2: 비디오와 오디오 결합
    video_cmd = [
        'ffmpeg', '-y',
        '-i', str(original_video_path),
        '-i', str(mixed_audio_path),
        '-map', '0:v:0',
        '-map', '1:a:0',
        '-c:v', 'libx264',
        '-preset', 'medium',
        '-crf', '23',
        '-c:a', 'aac',
        '-b:a', '192k',
        '-shortest',
        str(output_video_path)
    ]
    
    print("[STEP 2] 🎬 비디오 인코딩 중...")
    
    result = subprocess.run(video_cmd, capture_output=True, encoding='utf-8', errors='replace')
    
    if result.returncode != 0:
        print(f"\n❌ 비디오 인코딩 실패!")
        print(f"Return code: {result.returncode}")
        print(f"\nStderr:\n{result.stderr}")
        raise subprocess.CalledProcessError(result.returncode, video_cmd, result.stdout, result.stderr)
    
    # 임시 오디오 파일 삭제
    try:
        mixed_audio_path.unlink()
        print(f"[CLEANUP] 🗑️  임시 파일 삭제 완료")
    except:
        pass
    
    print(f"[STEP 2] ✅ 비디오 인코딩 완료!")
    return output_video_path

# 최종 출력 영상 경로
final_video_path = OUTPUT_VIDEO_DIR / f"{video_stem}.final.mp4"

print("\n" + "=" * 80)
print("🎬 최종 영상 생성 시작")
print("=" * 80)
print(f"  📹 원본 비디오: {local_video_path.name}")
print(f"  🎤 TTS 음성: {final_tts_wav.name}")
print(f"  🎵 배경음: {no_vocals_path.name}")
print(f"  💾 출력: {final_video_path.name}")
print("=" * 80)

try:
    final_video = merge_audio_and_encode_video(
        original_video_path=local_video_path,
        tts_vocals_path=final_tts_wav,
        bg_no_vocals_path=no_vocals_path,
        output_video_path=final_video_path,
        tts_volume=1.0,
        bg_volume=0.7,
    )
    
    print("\n" + "="*80)
    print("🎉 전체 파이프라인 완료!")
    print("="*80)
    print(f"✅ 최종 영상: {final_video}")
    print(f"📦 파일 크기: {final_video.stat().st_size / (1024**2):.2f} MB")
    
    # 최종 영상 길이 확인
    try:
        final_duration = get_duration(final_video)
        print(f"⏱️  영상 길이: {final_duration:.2f}초 ({int(final_duration//60)}분 {int(final_duration%60)}초)")
    except:
        pass
    
    print("="*80 + "\n")
    
except subprocess.CalledProcessError as e:
    print(f"\n❌ FFmpeg 인코딩 실패!")
    print(f"에러 코드: {e.returncode}")
    if e.stderr:
        print(f"\n상세 에러 메시지:")
        print("=" * 80)
        print(e.stderr)
        print("=" * 80)
    raise
    
except Exception as e:
    print(f"\n❌ 최종 인코딩 실패: {e}")
    import traceback
    traceback.print_exc()
    raise